#### Quarterly Sales Tax Revenue Analysis

### [Data Source](https://tax.utah.gov/econstats/sales)

### Q4: October, November, December

### Notes:
- Q4 2020 excel sheet does not contain totals for the 'UNKNOWN/NONCLASSIFIABLE' category; respective column for 2019 was removed

In [1]:
import pandas as pd
import os
import numpy as np
import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

In [2]:
# show all columns
pd.options.display.max_columns = None

In [3]:
# Removes misc characters from string
def replace(string):
    string = string.replace("&", "")
    string = string.replace("-", " ")
    string = string.replace("(", "")
    string = string.replace(")", "")
    string = string.replace("/", " ")
    string = string.replace(",", "")
    string = string.replace(".", "")
    string = string.replace(" ", "_")
    return string


### Load Q4
2020 sales tax revenue data

In [4]:
# load excel sheet into a dataframe
xlsx = '.\\Inputs\\2020-04-quarterly-sales.xlsx'
q4_2020 = pd.read_excel(xlsx, sheet_name='Table 9', header=5)

# Drop last empty two rows
q4_2020.drop(q4_2020.tail(2).index,inplace=True)

# forward fill values from merged cells
q4_2020['County'] = q4_2020['County'].fillna(method='ffill')
q4_2020['Location Code'] = q4_2020['Location Code'].fillna(method='ffill')
q4_2020['City'] = q4_2020['City'].fillna(method='ffill')

# Figures with less than 10 taxpayers have been rounded up per Tax Commission disclosure rules.
# Rename column and convert values to boolean
q4_2020.rename(columns={'Unnamed: 5':'Rounded Up'}, inplace=True)
crosswalk = {'*':True, np.nan:False} # County_ID
q4_2020['Rounded Up'] = q4_2020['Rounded Up'].map(crosswalk)

q4_2020.head()


,County,Location Code,City,Economic Sector (NAICS Code),2020Q4,Rounded Up
0,Beaver County,1002.0,Beaver City,"AGRICULTURE, FORESTRY, FISHING & HUNTING (11)",30000.0,True
1,Beaver County,1002.0,Beaver City,UTILITIES (22),700000.0,True
2,Beaver County,1002.0,Beaver City,CONSTRUCTION (23),93769.0,False
3,Beaver County,1002.0,Beaver City,MANUFACTURING (31-33),1909328.0,False
4,Beaver County,1002.0,Beaver City,WHOLESALE TRADE-DURABLE GOODS (423),317274.0,False


In [5]:
# pivot the table to get sectors as columns with tax values by city
sectors_q4_2020 = pd.pivot_table(q4_2020,values = '2020Q4',index ='City', columns = 'Economic Sector (NAICS Code)', 
                                 aggfunc='first')

# fill NAs with 0
sectors_q4_2020 = sectors_q4_2020.fillna(0)

In [6]:
sectors_q4_2020.columns

Index(['ACCOMMODATION (721)',
       'ADMIN. & SUPPORT & WASTE MANAG. & REMED. SERVICES (56)',
       'AGRICULTURE, FORESTRY, FISHING & HUNTING (11)',
       'ARTS, ENTERTAINMENT AND RECREATION (71)', 'CONSTRUCTION (23)',
       'EDUCATIONAL SERVICES (61)', 'FINANCE & INSURANCE (52)',
       'FOOD SERVICES & DRINKING PLACES (722)',
       'HEALTH CARE & SOCIAL ASSISTANCE (62)', 'INFORMATION (51)',
       'MANAGEMENT OF COMPANIES & ENTERPRISES (55)', 'MANUFACTURING (31-33)',
       'MINING, QUARRYING, & OIL & GAS EXTRACTION (21)',
       'OTHER SERVICES-EXCEPT PUBLIC ADMINISTRATION (81)',
       'PRIOR-PERIOD PAYMENTS & REFUNDS', 'PRIVATE MOTOR VEHICLE SALES',
       'PROFESSIONAL, SCIENTIFIC & TECHNICAL SERVICES (54)',
       'PUBLIC ADMINISTRATION (92)', 'REAL ESTATE, RENTAL & LEASING (53)',
       'RETAIL-BUILD. MATERIAL, GARDEN EQUIP. & SUPPLIES DEALERS (444)',
       'RETAIL-CLOTHING & CLOTHING ACCESSORIES STORES (448)',
       'RETAIL-ELECTRONICS & APPLIANCE STORES (443)',
       

In [7]:
# fix column names
#new_names = [replace(item) for item in list(sectors_q4_2020.columns)]
new_names = ['L_ACCOMMODATION_20', 'S_ADMIN_SUPPORT_20','A_AG_WILDLIFE_20','L_CULTURAL_REC_20','C_CONSTRUCTION_20',
'E_EDUCATION_20','S_FINANCIAL_20','L_RSTRNT_BAR_20','H_HEALTH_CARE_20','S_IT_DATA_20','S_CORPORATE_MGMT_20',
'M_MANUFACTURING_20','E_EXTRACTION_20','O_OTHER_20','X_ADJUSTMENTS_20','R_AUTO_PRIVATE_20','S_PROF_TECH_SRV_20',
'G_GOVERNMENT_20','S_RENTAL_LEASING_20','R_BUILDING_SUPPLY_20','R_CLOTHING_20','R_ELECTRONICS_20','R_GROCERY_BEV_20',
'R_FURNITURE_20','R_GAS_STATIONS_20','R_GENERAL_RETAIL_20','R_HEALTH_RETAIL_20','R_OTHER_RETAIL_20','R_AUTO_RETAIL_20',
'R_NONSTORE_RETAIL_20','R_SPORT_HOBBY_20','L_SPECIAL_EVENT_20','W_DISTRIBUTION_20','X_UNKNOWN_20','U_UTILITIES_20',
'W_WHLSALE_DURABLE_20','S_WHLSLE_ETRADE_20','W_WHLSLE_NDURABLE_20']
sectors_q4_2020.columns = new_names

# get the total sales tax revenue
sectors_q4_2020['TOTAL_20'] = sectors_q4_2020[new_names].sum(axis=1)

sectors_q4_2020.head()

,L_ACCOMMODATION_20,S_ADMIN_SUPPORT_20,A_AG_WILDLIFE_20,L_CULTURAL_REC_20,C_CONSTRUCTION_20,E_EDUCATION_20,S_FINANCIAL_20,L_RSTRNT_BAR_20,H_HEALTH_CARE_20,S_IT_DATA_20,S_CORPORATE_MGMT_20,M_MANUFACTURING_20,E_EXTRACTION_20,O_OTHER_20,X_ADJUSTMENTS_20,R_AUTO_PRIVATE_20,S_PROF_TECH_SRV_20,G_GOVERNMENT_20,S_RENTAL_LEASING_20,R_BUILDING_SUPPLY_20,R_CLOTHING_20,R_ELECTRONICS_20,R_GROCERY_BEV_20,R_FURNITURE_20,R_GAS_STATIONS_20,R_GENERAL_RETAIL_20,R_HEALTH_RETAIL_20,R_OTHER_RETAIL_20,R_AUTO_RETAIL_20,R_NONSTORE_RETAIL_20,R_SPORT_HOBBY_20,L_SPECIAL_EVENT_20,W_DISTRIBUTION_20,X_UNKNOWN_20,U_UTILITIES_20,W_WHLSALE_DURABLE_20,S_WHLSLE_ETRADE_20,W_WHLSLE_NDURABLE_20,TOTAL_20
City,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
American Fork,445919.0,946108.0,400000.0,1383561.0,11147175.0,98622.0,3568796.0,27448933.0,649328.0,9578219.0,25000.0,3582969.0,150000.0,12001800.0,4157186.0,6500000.0,7326059.0,20000.0,3690672.0,23190264.0,9857552.0,9775817.0,3874635.0,5670578.0,2750000.0,55836489.0,9025872.0,9104334.0,104582789.0,26428326.0,12945079.0,150000.0,540533.0,0.0,6500000.0,16988388.0,595216.0,7380982.0,398317201.0
Beaver City,1263540.0,21241.0,30000.0,8000.0,93769.0,20000.0,515183.0,2642442.0,60000.0,1475591.0,1000.0,1909328.0,0.0,928671.0,215431.0,400000.0,175210.0,0.0,196120.0,873826.0,199170.0,41237.0,2000000.0,33318.0,3750000.0,577581.0,120653.0,221017.0,728301.0,1978122.0,74066.0,15000.0,2000.0,0.0,700000.0,317274.0,67452.0,171669.0,21826212.0
Blanding,674993.0,24381.0,1000.0,45000.0,212049.0,70000.0,17960.0,400000.0,40000.0,434186.0,1000.0,409957.0,0.0,456694.0,-4831.0,700000.0,158873.0,500000.0,66774.0,2255336.0,121948.0,108118.0,2000000.0,41293.0,1750000.0,777912.0,101531.0,150551.0,400673.0,1999356.0,96034.0,0.0,20000.0,0.0,90000.0,408884.0,7826.0,785640.0,15323138.0
Bountiful,200000.0,1670480.0,7550.0,1218068.0,1718083.0,89674.0,1139404.0,13172819.0,1623303.0,6279817.0,20000.0,3261099.0,40000.0,6319648.0,1789498.0,4250000.0,4820458.0,300000.0,2233518.0,2865086.0,4175511.0,5053796.0,29675024.0,2085829.0,2500000.0,3210434.0,2494648.0,4775024.0,50890289.0,24458264.0,2527673.0,30000.0,27650.0,0.0,7750000.0,6783633.0,129876.0,1039011.0,200625167.0
Brigham,905882.0,196396.0,2000.0,311891.0,921313.0,68816.0,232119.0,7240559.0,433033.0,3040482.0,4000.0,8030351.0,8000.0,3697620.0,537328.0,2000000.0,925025.0,1000.0,2226479.0,3503291.0,717404.0,1773289.0,13453415.0,641342.0,3750000.0,989161.0,405695.0,1085983.0,15941089.0,8518040.0,747995.0,0.0,112969.0,0.0,3500000.0,6123833.0,156823.0,389195.0,92591818.0


#### Load Q4 2019 sales tax revenue data

In [8]:
# load excel sheet into a dataframe
xlsx = '.\\Inputs\\2019-04-quarterly-sales.xlsx'
q4_2019 = pd.read_excel(xlsx, sheet_name='Table 9', header=5)

# Drop last empty two rows
q4_2019.drop(q4_2019.tail(2).index,inplace=True)

# forward fill values from merged cells
q4_2019['County'] = q4_2019['County'].fillna(method='ffill')
q4_2019['Location Code'] = q4_2019['Location Code'].fillna(method='ffill')
q4_2019['City'] = q4_2019['City'].fillna(method='ffill')

# Figures with less than 10 taxpayers have been rounded up per Tax Commission disclosure rules.
# Rename column and convert values to boolean
q4_2019.rename(columns={'Unnamed: 5':'Rounded Up'}, inplace=True)
crosswalk = {'*':True, np.nan:False} # County_ID
q4_2019['Rounded Up'] = q4_2019['Rounded Up'].map(crosswalk)

q4_2019.tail()

,County,Location Code,City,Economic Sector (NAICS Code),2019Q4,Rounded Up
2410,Weber County,29040.0,South Ogden,OTHER SERVICES-EXCEPT PUBLIC ADMINISTRATION (81),1391763.0,False
2411,Weber County,29040.0,South Ogden,PUBLIC ADMINISTRATION (92),1000.0,True
2412,Weber County,29040.0,South Ogden,PRIVATE MOTOR VEHICLE SALES,1500000.0,True
2413,Weber County,29040.0,South Ogden,SPECIAL EVENT SALES,35000.0,True
2414,Weber County,29040.0,South Ogden,PRIOR-PERIOD PAYMENTS & REFUNDS,179101.0,False


In [13]:
# pivot the table to get sectors as columns with tax values by city
sectors_q4_2019 = pd.pivot_table(q4_2019,values = '2019Q4',index ='City', columns = 'Economic Sector (NAICS Code)', 
                                 aggfunc='first')

# fill NAs with 0
sectors_q4_2019 = sectors_q4_2019.fillna(0)

# fix column names
#new_names = [replace(item) for item in list(sectors_q4_2019.columns)]
new_names = ['L_ACCOMMODATION_19', 'S_ADMIN_SUPPORT_19','A_AG_WILDLIFE_19','L_CULTURAL_REC_19','C_CONSTRUCTION_19',
'E_EDUCATION_19','S_FINANCIAL_19','L_RSTRNT_BAR_19','H_HEALTH_CARE_19','S_IT_DATA_19','S_CORPORATE_MGMT_19',
'M_MANUFACTURING_19','E_EXTRACTION_19','O_OTHER_19','X_ADJUSTMENTS_19','R_AUTO_PRIVATE_19','S_PROF_TECH_SRV_19',
'G_GOVERNMENT_19','S_RENTAL_LEASING_19','R_BUILDING_SUPPLY_19','R_CLOTHING_19','R_ELECTRONICS_19','R_GROCERY_BEV_19',
'R_FURNITURE_19','R_GAS_STATIONS_19','R_GENERAL_RETAIL_19','R_HEALTH_RETAIL_19','R_OTHER_RETAIL_19','R_AUTO_RETAIL_19',
'R_NONSTORE_RETAIL_19','R_SPORT_HOBBY_19','L_SPECIAL_EVENT_19','W_DISTRIBUTION_19','X_UNKNOWN_19','U_UTILITIES_19',
'W_WHLSALE_DURABLE_19','S_WHLSLE_ETRADE_19','W_WHLSLE_NDURABLE_19']
sectors_q4_2019.columns = new_names

# drop unknown column since its not present in 2020
#sectors_q4_2019.drop('X_UNKNOWN_19', axis=1, inplace=True)
#new_names.remove('X_UNKNOWN_19')

# get the total sales tax revenue
sectors_q4_2019['TOTAL_19'] = sectors_q4_2019[new_names].sum(axis=1)

sectors_q4_2019.head()

,L_ACCOMMODATION_19,S_ADMIN_SUPPORT_19,A_AG_WILDLIFE_19,L_CULTURAL_REC_19,C_CONSTRUCTION_19,E_EDUCATION_19,S_FINANCIAL_19,L_RSTRNT_BAR_19,H_HEALTH_CARE_19,S_IT_DATA_19,S_CORPORATE_MGMT_19,M_MANUFACTURING_19,E_EXTRACTION_19,O_OTHER_19,X_ADJUSTMENTS_19,R_AUTO_PRIVATE_19,S_PROF_TECH_SRV_19,G_GOVERNMENT_19,S_RENTAL_LEASING_19,R_BUILDING_SUPPLY_19,R_CLOTHING_19,R_ELECTRONICS_19,R_GROCERY_BEV_19,R_FURNITURE_19,R_GAS_STATIONS_19,R_GENERAL_RETAIL_19,R_HEALTH_RETAIL_19,R_OTHER_RETAIL_19,R_AUTO_RETAIL_19,R_NONSTORE_RETAIL_19,R_SPORT_HOBBY_19,L_SPECIAL_EVENT_19,W_DISTRIBUTION_19,X_UNKNOWN_19,U_UTILITIES_19,W_WHLSALE_DURABLE_19,S_WHLSLE_ETRADE_19,W_WHLSLE_NDURABLE_19,TOTAL_19
City,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
American Fork,554928.0,698435.0,200000.0,1565976.0,9116585.0,58968.0,4184777.0,24894878.0,553191.0,10533285.0,9000.0,2593500.0,80000.0,10867304.0,3517109.0,2500000.0,4072879.0,15000.0,3694025.0,18791011.0,9024897.0,11596032.0,4074889.0,4627530.0,2695622.0,50966283.0,6437283.0,7076800.0,94459123.0,19598548.0,10893808.0,200000.0,600000.0,0.0,6250000.0,12675770.0,23966.0,5379266.0,345080668.0
Beaver City,1056763.0,20808.0,15000.0,9000.0,70000.0,25000.0,515603.0,2206640.0,45000.0,520147.0,3000.0,1497139.0,0.0,782177.0,-555466.0,300000.0,123929.0,0.0,110090.0,566512.0,117279.0,30530.0,1750000.0,17014.0,3250000.0,531241.0,109923.0,579869.0,787860.0,1566737.0,55127.0,6000.0,8000.0,0.0,700000.0,425319.0,20000.0,227851.0,17494092.0
Blanding,761898.0,61901.0,1000.0,70000.0,276229.0,150000.0,21221.0,450000.0,60000.0,431669.0,4000.0,94027.0,0.0,277858.0,1129905.0,600000.0,152346.0,600000.0,74215.0,1717197.0,83400.0,143702.0,1750000.0,8692.0,1250000.0,690374.0,92246.0,155179.0,304971.0,1538996.0,50048.0,4000.0,40000.0,0.0,100000.0,343209.0,4000.0,255351.0,13747634.0
Bountiful,150000.0,2291194.0,2000.0,1098539.0,1894207.0,91173.0,1332378.0,12786832.0,1039103.0,5303124.0,9000.0,2863668.0,15000.0,6239951.0,2904446.0,3250000.0,4140342.0,250000.0,2171159.0,1061688.0,4059340.0,3841132.0,28438413.0,1633886.0,2250000.0,2749493.0,2163769.0,3897805.0,50627772.0,18327977.0,2197872.0,30000.0,45000.0,0.0,7500000.0,5143867.0,22145.0,737898.0,182560173.0
Brigham,984729.0,222487.0,1000.0,260399.0,625648.0,67697.0,455220.0,6659356.0,308659.0,2858043.0,25000.0,6604323.0,1000.0,3293364.0,-1541163.0,1500000.0,870400.0,1000.0,1395475.0,1714961.0,583848.0,1520067.0,12372772.0,525222.0,3046042.0,900956.0,363253.0,1217788.0,14838915.0,6501460.0,545826.0,15000.0,64106.0,0.0,3500000.0,4773142.0,15000.0,394082.0,77485077.0


### Calculate the differences 
*2020 - 2019 = Diff*

In [14]:
# Check join output shape
print(sectors_q4_2019.shape)
print(sectors_q4_2020.shape)

(66, 39)
(66, 39)


In [15]:
# merge the 2019-2020 difference with the sdf
sectors_q4_diff = pd.DataFrame(sectors_q4_2020.values - sectors_q4_2019.values)

# add back the column names
new_names = ['L_ACCOMMODATION_D20', 'S_ADMIN_SUPPORT_D20','A_AG_WILDLIFE_D20','L_CULTURAL_REC_D20','C_CONSTRUCTION_D20',
'E_EDUCATION_D20','S_FINANCIAL_D20','L_RSTRNT_BAR_D20','H_HEALTH_CARE_D20','S_IT_DATA_D20','S_CORPORATE_MGMT_D20',
'M_MANUFACTURING_D20','E_EXTRACTION_D20','O_OTHER_D20','X_ADJUSTMENTS_D20','R_AUTO_PRIVATE_D20','S_PROF_TECH_SRV_D20',
'G_GOVERNMENT_D20','S_RENTAL_LEASING_D20','R_BUILDING_SUPPLY_D20','R_CLOTHING_D20','R_ELECTRONICS_D20','R_GROCERY_BEV_D20',
'R_FURNITURE_D20','R_GAS_STATIONS_D20','R_GENERAL_RETAIL_D20','R_HEALTH_RETAIL_D20','R_OTHER_RETAIL_D20','R_AUTO_RETAIL_D20',
'R_NONSTORE_RETAIL_D20','R_SPORT_HOBBY_D20','L_SPECIAL_EVENT_D20','W_DISTRIBUTION_D20','X_UNKNOWN_D20','U_UTILITIES_D20',
'W_WHLSALE_DURABLE_D20','S_WHLSLE_ETRADE_D20','W_WHLSLE_NDURABLE_D20', 'TOTAL_D20']

sectors_q4_diff.columns = new_names

# add back cities as index
sectors_q4_diff['City'] = sectors_q4_2020.index
sectors_q4_diff = sectors_q4_diff.set_index('City')

#check table
sectors_q4_diff.head(10)

,L_ACCOMMODATION_D20,S_ADMIN_SUPPORT_D20,A_AG_WILDLIFE_D20,L_CULTURAL_REC_D20,C_CONSTRUCTION_D20,E_EDUCATION_D20,S_FINANCIAL_D20,L_RSTRNT_BAR_D20,H_HEALTH_CARE_D20,S_IT_DATA_D20,S_CORPORATE_MGMT_D20,M_MANUFACTURING_D20,E_EXTRACTION_D20,O_OTHER_D20,X_ADJUSTMENTS_D20,R_AUTO_PRIVATE_D20,S_PROF_TECH_SRV_D20,G_GOVERNMENT_D20,S_RENTAL_LEASING_D20,R_BUILDING_SUPPLY_D20,R_CLOTHING_D20,R_ELECTRONICS_D20,R_GROCERY_BEV_D20,R_FURNITURE_D20,R_GAS_STATIONS_D20,R_GENERAL_RETAIL_D20,R_HEALTH_RETAIL_D20,R_OTHER_RETAIL_D20,R_AUTO_RETAIL_D20,R_NONSTORE_RETAIL_D20,R_SPORT_HOBBY_D20,L_SPECIAL_EVENT_D20,W_DISTRIBUTION_D20,X_UNKNOWN_D20,U_UTILITIES_D20,W_WHLSALE_DURABLE_D20,S_WHLSLE_ETRADE_D20,W_WHLSLE_NDURABLE_D20,TOTAL_D20
City,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
American Fork,-109009.0,247673.0,200000.0,-182415.0,2030590.0,39654.0,-615981.0,2554055.0,96137.0,-955066.0,16000.0,989469.0,70000.0,1134496.0,640077.0,4000000.0,3253180.0,5000.0,-3353.0,4399253.0,832655.0,-1820215.0,-200254.0,1043048.0,54378.0,4870206.0,2588589.0,2027534.0,10123666.0,6829778.0,2051271.0,-50000.0,-59467.0,0.0,250000.0,4312618.0,571250.0,2001716.0,53236533.0
Beaver City,206777.0,433.0,15000.0,-1000.0,23769.0,-5000.0,-420.0,435802.0,15000.0,955444.0,-2000.0,412189.0,0.0,146494.0,770897.0,100000.0,51281.0,0.0,86030.0,307314.0,81891.0,10707.0,250000.0,16304.0,500000.0,46340.0,10730.0,-358852.0,-59559.0,411385.0,18939.0,9000.0,-6000.0,0.0,0.0,-108045.0,47452.0,-56182.0,4332120.0
Blanding,-86905.0,-37520.0,0.0,-25000.0,-64180.0,-80000.0,-3261.0,-50000.0,-20000.0,2517.0,-3000.0,315930.0,0.0,178836.0,-1134736.0,100000.0,6527.0,-100000.0,-7441.0,538139.0,38548.0,-35584.0,250000.0,32601.0,500000.0,87538.0,9285.0,-4628.0,95702.0,460360.0,45986.0,-4000.0,-20000.0,0.0,-10000.0,65675.0,3826.0,530289.0,1575504.0
Bountiful,50000.0,-620714.0,5550.0,119529.0,-176124.0,-1499.0,-192974.0,385987.0,584200.0,976693.0,11000.0,397431.0,25000.0,79697.0,-1114948.0,1000000.0,680116.0,50000.0,62359.0,1803398.0,116171.0,1212664.0,1236611.0,451943.0,250000.0,460941.0,330879.0,877219.0,262517.0,6130287.0,329801.0,0.0,-17350.0,0.0,250000.0,1639766.0,107731.0,301113.0,18064994.0
Brigham,-78847.0,-26091.0,1000.0,51492.0,295665.0,1119.0,-223101.0,581203.0,124374.0,182439.0,-21000.0,1426028.0,7000.0,404256.0,2078491.0,500000.0,54625.0,0.0,831004.0,1788330.0,133556.0,253222.0,1080643.0,116120.0,703958.0,88205.0,42442.0,-131805.0,1102174.0,2016580.0,202169.0,-15000.0,48863.0,0.0,0.0,1350691.0,141823.0,-4887.0,15106741.0
Cedar City,-41175.0,671086.0,-4710.0,114423.0,385898.0,-148550.0,77353.0,2374878.0,5390.0,-74291.0,-1000.0,269091.0,0.0,1894751.0,-28720728.0,1750000.0,363599.0,0.0,827165.0,12774120.0,933661.0,1966278.0,3690609.0,2033933.0,1384651.0,4079159.0,141769.0,604511.0,4261673.0,4558936.0,2744426.0,-30000.0,-62337.0,0.0,250000.0,5361870.0,209906.0,-525951.0,24120394.0
Centerville,25000.0,48956.0,-3000.0,55000.0,-236875.0,32029.0,-50352.0,413992.0,46730.0,-1913699.0,0.0,811420.0,0.0,1031207.0,257292.0,500000.0,363776.0,0.0,171520.0,3541887.0,213491.0,-238309.0,426746.0,-617883.0,250000.0,739855.0,46119.0,-1104125.0,-5533468.0,2415481.0,209630.0,0.0,0.0,0.0,0.0,593149.0,334198.0,-19552.0,2810215.0
Clearfield,50000.0,-233464.0,2000.0,-373752.0,534101.0,9142.0,-86233.0,391381.0,-6320.0,1116903.0,200000.0,-2356535.0,-5000.0,354790.0,-2098300.0,0.0,122200.0,50000.0,-180541.0,-27387.0,533989.0,9902.0,519777.0,-1406.0,893480.0,309357.0,153024.0,301936.0,422462.0,5145395.0,-56020.0,0.0,-10000.0,0.0,-1500000.0,366243.0,218434.0,-258234.0,4511324.0
Cottonwood Heights,129633.0,-20690.0,2000.0,-742029.0,-733031.0,-78076.0,7387.0,-684317.0,-50553.0,928925.0,-100000.0,-397061.0,0.0,-33830.0,1143944.0,500000.0,2302578.0,-100000.0,-151370.0,2535675.0,664285.0,540777.0,1419321.0,167446.0,0.0,4432611.0,684683.0,1087538.0,-240159.0,4825050.0,710298.0,-60000.0,5000.0,0.0,500000.0,1048872.0,93931.0,289110.0,20627948.0


### Load municipalities and townships shapefile and join to formatted sales tax data

In [16]:
# load cities shapefile into pandas spatial dataframe
cities_shp = '.\\Inputs\\Cities.shp'
cities_sdf = pd.DataFrame.spatial.from_featureclass(cities_shp)

# Rename columns
cities_sdf.columns = ['FID', 'NAME', 'SOURCE', 'SALESTAXID', 'POPLASTCEN', 'POPLASTEST',
       'AREA_SQMI', 'SHAPE']

In [17]:
# merge the 2020 sdf with the pivoted data
cities_sdf2 = cities_sdf.merge(sectors_q4_2019, left_on= "NAME", right_on="City", how="inner")
cities_sdf2 = cities_sdf2.merge(sectors_q4_2020, left_on= "NAME", right_on="City", how="inner")
cities_sdf2 = cities_sdf2.merge(sectors_q4_diff, left_on= "NAME", right_on="City", how="inner")
cities_sdf2.tail(10)


,FID,NAME,SOURCE,SALESTAXID,POPLASTCEN,POPLASTEST,AREA_SQMI,SHAPE,L_ACCOMMODATION_19,S_ADMIN_SUPPORT_19,A_AG_WILDLIFE_19,L_CULTURAL_REC_19,C_CONSTRUCTION_19,E_EDUCATION_19,S_FINANCIAL_19,L_RSTRNT_BAR_19,H_HEALTH_CARE_19,S_IT_DATA_19,S_CORPORATE_MGMT_19,M_MANUFACTURING_19,E_EXTRACTION_19,O_OTHER_19,X_ADJUSTMENTS_19,R_AUTO_PRIVATE_19,S_PROF_TECH_SRV_19,G_GOVERNMENT_19,S_RENTAL_LEASING_19,R_BUILDING_SUPPLY_19,R_CLOTHING_19,R_ELECTRONICS_19,R_GROCERY_BEV_19,R_FURNITURE_19,R_GAS_STATIONS_19,R_GENERAL_RETAIL_19,R_HEALTH_RETAIL_19,R_OTHER_RETAIL_19,R_AUTO_RETAIL_19,R_NONSTORE_RETAIL_19,R_SPORT_HOBBY_19,L_SPECIAL_EVENT_19,W_DISTRIBUTION_19,X_UNKNOWN_19,U_UTILITIES_19,W_WHLSALE_DURABLE_19,S_WHLSLE_ETRADE_19,W_WHLSLE_NDURABLE_19,TOTAL_19,L_ACCOMMODATION_20,S_ADMIN_SUPPORT_20,A_AG_WILDLIFE_20,L_CULTURAL_REC_20,C_CONSTRUCTION_20,E_EDUCATION_20,S_FINANCIAL_20,L_RSTRNT_BAR_20,H_HEALTH_CARE_20,S_IT_DATA_20,S_CORPORATE_MGMT_20,M_MANUFACTURING_20,E_EXTRACTION_20,O_OTHER_20,X_ADJUSTMENTS_20,R_AUTO_PRIVATE_20,S_PROF_TECH_SRV_20,G_GOVERNMENT_20,S_RENTAL_LEASING_20,R_BUILDING_SUPPLY_20,R_CLOTHING_20,R_ELECTRONICS_20,R_GROCERY_BEV_20,R_FURNITURE_20,R_GAS_STATIONS_20,R_GENERAL_RETAIL_20,R_HEALTH_RETAIL_20,R_OTHER_RETAIL_20,R_AUTO_RETAIL_20,R_NONSTORE_RETAIL_20,R_SPORT_HOBBY_20,L_SPECIAL_EVENT_20,W_DISTRIBUTION_20,X_UNKNOWN_20,U_UTILITIES_20,W_WHLSALE_DURABLE_20,S_WHLSLE_ETRADE_20,W_WHLSLE_NDURABLE_20,TOTAL_20,L_ACCOMMODATION_D20,S_ADMIN_SUPPORT_D20,A_AG_WILDLIFE_D20,L_CULTURAL_REC_D20,C_CONSTRUCTION_D20,E_EDUCATION_D20,S_FINANCIAL_D20,L_RSTRNT_BAR_D20,H_HEALTH_CARE_D20,S_IT_DATA_D20,S_CORPORATE_MGMT_D20,M_MANUFACTURING_D20,E_EXTRACTION_D20,O_OTHER_D20,X_ADJUSTMENTS_D20,R_AUTO_PRIVATE_D20,S_PROF_TECH_SRV_D20,G_GOVERNMENT_D20,S_RENTAL_LEASING_D20,R_BUILDING_SUPPLY_D20,R_CLOTHING_D20,R_ELECTRONICS_D20,R_GROCERY_BEV_D20,R_FURNITURE_D20,R_GAS_STATIONS_D20,R_GENERAL_RETAIL_D20,R_HEALTH_RETAIL_D20,R_OTHER_RETAIL_D20,R_AUTO_RETAIL_D20,R_NONSTORE_RETAIL_D20,R_SPORT_HOBBY_D20,L_SPECIAL_EVENT_D20,W_DISTRIBUTION_D20,X_UNKNOWN_D20,U_UTILITIES_D20,W_WHLSALE_DURABLE_D20,S_WHLSLE_ETRADE_D20,W_WHLSLE_NDURABLE_D20,TOTAL_D20
55,222,Syracuse,Cities,049,25113,30400,10.193900,"{'rings': [[[411406.60675760085, 4549994.30436...",35000.0,83233.0,1000.0,1006625.0,215900.0,17715.0,419955.0,4513352.0,24345.0,3145080.0,40000.0,469106.0,15000.0,1576476.0,1120174.0,2250000.0,782275.0,15000.0,905855.0,523754.0,713985.0,1044991.0,11657032.0,11346841.0,450000.0,21121201.0,473430.0,773584.0,470211.0,11881965.0,271073.0,35000.0,6000.0,0.0,4000000.0,1332461.0,15000.0,324344.0,83076963.0,60000.0,129870.0,2682.0,1015951.0,442050.0,20052.0,469486.0,5151861.0,23686.0,3350336.0,2000.0,1219870.0,25000.0,1511565.0,1042832.0,3500000.0,1096230.0,6000.0,1004668.0,1313864.0,1124319.0,1407697.0,13792317.0,505966.0,500000.0,23612809.0,593798.0,894001.0,631857.0,16920895.0,456660.0,10000.0,50000.0,0.0,4250000.0,2930106.0,156749.0,481130.0,89706307.0,25000.0,46637.0,1682.0,9326.0,226150.0,2337.0,49531.0,638509.0,-659.0,205256.0,-38000.0,750764.0,10000.0,-64911.0,-77342.0,1250000.0,313955.0,-9000.0,98813.0,790110.0,410334.0,362706.0,2135285.0,-10840875.0,50000.0,2491608.0,120368.0,120417.0,161646.0,5038930.0,185587.0,-25000.0,44000.0,0.0,250000.0,1597645.0,141749.0,156786.0,6629344.0
56,224,Taylorsville,Cities,142,59989,60192,10.839700,"{'rings': [[[422191.826826254, 4504303.3642982...",350000.0,448861.0,15000.0,1363787.0,1374212.0,115555.0,2884476.0,29151676.0,147429.0,10615060.0,9000.0,2637643.0,20000.0,2023233.0,1477947.0,2750000.0,2521638.0,3000.0,3287932.0,545764.0,7628480.0,2561020.0,15759778.0,431915.0,3750000.0,30079535.0,1332151.0,4861139.0,6733275.0,15841523.0,8110273.0,10000.0,50000.0,0.0,9250000.0,3650755.0,35000.0,480441.0,172307498.0,300000.0,493502.0,12414.0,1114428.0,1629065.0,91835.0,2253397.0,29773430.0,224421.0,7384084.0,8000.0,3704083.0,20000.0,2082745.0,2279191.0,4750000.0,3200452.0,2000.0,1980511.0,584442.0,7494416.0,3824530.0,15488549.0,484443.0,4250000.0,38047508.0,1394520.0,61

In [18]:
# Confirm join count, Salt Lake County (Unincorporated) should be the only one not joined
l1 = list(q4_2020['City'].value_counts().index)
l2 = list(cities_sdf['NAME'].value_counts().index)
list(set(l1) - set(l2))

['Salt Lake County (Unincorporated)']

In [19]:
# Create file gdb and export sdf to feature class
outputs = '.\\Outputs'

gdb = os.path.join(outputs, "taxable_sales.gdb")
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs, "taxable_sales.gdb")

cities_sdf2.spatial.to_featureclass(location=os.path.join(outputs, "taxable_sales.gdb","taxable_sales_utah_q4"))

'C:\\LocalGIS\\GitProjects\\UT_Sales_Tax_Analysis\\Outputs\\taxable_sales.gdb\\taxable_sales_utah_q4'

### Analysis

In [21]:
cities_sdf2

,FID,NAME,SOURCE,SALESTAXID,POPLASTCEN,POPLASTEST,AREA_SQMI,SHAPE,L_ACCOMMODATION_19,S_ADMIN_SUPPORT_19,A_AG_WILDLIFE_19,L_CULTURAL_REC_19,C_CONSTRUCTION_19,E_EDUCATION_19,S_FINANCIAL_19,L_RSTRNT_BAR_19,H_HEALTH_CARE_19,S_IT_DATA_19,S_CORPORATE_MGMT_19,M_MANUFACTURING_19,E_EXTRACTION_19,O_OTHER_19,X_ADJUSTMENTS_19,R_AUTO_PRIVATE_19,S_PROF_TECH_SRV_19,G_GOVERNMENT_19,S_RENTAL_LEASING_19,R_BUILDING_SUPPLY_19,R_CLOTHING_19,R_ELECTRONICS_19,R_GROCERY_BEV_19,R_FURNITURE_19,R_GAS_STATIONS_19,R_GENERAL_RETAIL_19,R_HEALTH_RETAIL_19,R_OTHER_RETAIL_19,R_AUTO_RETAIL_19,R_NONSTORE_RETAIL_19,R_SPORT_HOBBY_19,L_SPECIAL_EVENT_19,W_DISTRIBUTION_19,X_UNKNOWN_19,U_UTILITIES_19,W_WHLSALE_DURABLE_19,S_WHLSLE_ETRADE_19,W_WHLSLE_NDURABLE_19,TOTAL_19,L_ACCOMMODATION_20,S_ADMIN_SUPPORT_20,A_AG_WILDLIFE_20,L_CULTURAL_REC_20,C_CONSTRUCTION_20,E_EDUCATION_20,S_FINANCIAL_20,L_RSTRNT_BAR_20,H_HEALTH_CARE_20,S_IT_DATA_20,S_CORPORATE_MGMT_20,M_MANUFACTURING_20,E_EXTRACTION_20,O_OTHER_20,X_ADJUSTMENTS_20,R_AUTO_PRIVATE_20,S_PROF_TECH_SRV_20,G_GOVERNMENT_20,S_RENTAL_LEASING_20,R_BUILDING_SUPPLY_20,R_CLOTHING_20,R_ELECTRONICS_20,R_GROCERY_BEV_20,R_FURNITURE_20,R_GAS_STATIONS_20,R_GENERAL_RETAIL_20,R_HEALTH_RETAIL_20,R_OTHER_RETAIL_20,R_AUTO_RETAIL_20,R_NONSTORE_RETAIL_20,R_SPORT_HOBBY_20,L_SPECIAL_EVENT_20,W_DISTRIBUTION_20,X_UNKNOWN_20,U_UTILITIES_20,W_WHLSALE_DURABLE_20,S_WHLSLE_ETRADE_20,W_WHLSLE_NDURABLE_20,TOTAL_20,L_ACCOMMODATION_D20,S_ADMIN_SUPPORT_D20,A_AG_WILDLIFE_D20,L_CULTURAL_REC_D20,C_CONSTRUCTION_D20,E_EDUCATION_D20,S_FINANCIAL_D20,L_RSTRNT_BAR_D20,H_HEALTH_CARE_D20,S_IT_DATA_D20,S_CORPORATE_MGMT_D20,M_MANUFACTURING_D20,E_EXTRACTION_D20,O_OTHER_D20,X_ADJUSTMENTS_D20,R_AUTO_PRIVATE_D20,S_PROF_TECH_SRV_D20,G_GOVERNMENT_D20,S_RENTAL_LEASING_D20,R_BUILDING_SUPPLY_D20,R_CLOTHING_D20,R_ELECTRONICS_D20,R_GROCERY_BEV_D20,R_FURNITURE_D20,R_GAS_STATIONS_D20,R_GENERAL_RETAIL_D20,R_HEALTH_RETAIL_D20,R_OTHER_RETAIL_D20,R_AUTO_RETAIL_D20,R_NONSTORE_RETAIL_D20,R_SPORT_HOBBY_D20,L_SPECIAL_EVENT_D20,W_DISTRIBUTION_D20,X_UNKNOWN_D20,U_UTILITIES_D20,W_WHLSALE_DURABLE_D20,S_WHLSLE_ETRADE_D20,W_WHLSLE_NDURABLE_D20,TOTAL_D20
0,5,American Fork,Cities,002,27413,32519,11.256600,"{""rings"": [[[434244.49682960496, 4467294.37416...",554928.0,698435.0,200000.0,1565976.0,9116585.0,58968.0,4184777.0,24894878.0,553191.0,10533285.0,9000.0,2593500.0,80000.0,10867304.0,3517109.0,2500000.0,4072879.0,15000.0,3694025.0,18791011.0,9024897.0,11596032.0,4074889.0,4627530.0,2695622.0,50966283.0,6437283.0,7076800.0,94459123.0,19598548.0,10893808.0,200000.0,600000.0,0.0,6250000.0,12675770.0,23966.0,5379266.0,345080668.0,445919.0,946108.0,400000.0,1383561.0,11147175.0,98622.0,3568796.0,27448933.0,649328.0,9578219.0,25000.0,3582969.0,150000.0,12001800.0,4157186.0,6500000.0,7326059.0,20000.0,3690672.0,23190264.0,9857552.0,9775817.0,3874635.0,5670578.0,2750000.0,55836489.0,9025872.0,9104334.0,104582789.0,26428326.0,12945079.0,150000.0,540533.0,0.0,6500000.0,16988388.0,595216.0,7380982.0,398317201.0,-109009.0,247673.0,200000.0,-182415.0,2030590.0,39654.0,-615981.0,2554055.0,96137.0,-955066.0,16000.0,989469.0,70000.0,1134496.0,640077.0,4000000.0,3253180.0,5000.0,-3353.0,4399253.0,832655.0,-1820215.0,-200254.0,1043048.0,54378.0,4870206.0,2588589.0,2027534.0,10123666.0,6829778.0,2051271.0,-50000.0,-59467.0,0.0,250000.0,4312618.0,571250.0,2001716.0,53236533.0
1,12,Beaver City,Cities,002,3137,3104,6.638160,"{""rings"": [[[355522.91657235345, 4239879.43388...",1056763.0,20808.0,15000.0,9000.0,70000.0,25000.0,515603.0,2206640.0,45000.0,520147.0,3000.0,1497139.0,0.0,782177.0,-555466.0,300000.0,123929.0,0.0,110090.0,566512.0,117279.0,30530.0,1750000.0,17014.0,3250000.0,531241.0,109923.0,579869.0,787860.0,1566737.0,55127.0,6000.0,8000.0,0.0,700000.0,425319.0,20000.0,227851.0,17494092.0,1263540.0,21241.0,30000.0,8000.0,93769.0,20000.0,515183.0,2642442.0,60000.0,1475591.0,1000.0,1909328.0,0.0,928671.0,215431.0,400000.0,175210.0,0.0,196120.0,873826.0,199170.0,41237.0,2000000.0,33318.0,3750000.0,577581.0,12

In [26]:
# Get year specific column names
standard_cols = ['FID','NAME','SOURCE','SALESTAXID','POPLASTCEN','POPLASTEST','AREA_SQMI', 'SHAPE']
names_19 = standard_cols + [col for col in cities_sdf2.columns if '_19' in col]
names_20 = standard_cols + [col for col in cities_sdf2.columns if '_20' in col]
names_D20 = standard_cols + [col for col in cities_sdf2.columns if '_D20' in col]

# subset columns by time
cities_q4_19 = cities_sdf2[names_19]
cities_q4_20 = cities_sdf2[names_20]
cities_q4_d20 = cities_sdf2[names_D20]


In [27]:
# 10 cities with smallest difference of revenue
cities_q4_d20[['NAME', 'POPLASTCEN', 'AREA_SQMI', 'TOTAL_D20']].sort_values('TOTAL_D20', ascending=False).head(10)

,NAME,POPLASTCEN,AREA_SQMI,TOTAL_D20
54,St George,76548,77.209900,151971179.0
62,West Jordan,105891,32.306999,83155206.0
20,Layton,69457,22.576200,79218781.0
34,Orem,92993,18.628000,70010105.0
33,Ogden,85183,27.493500,63608629.0
23,Logan,49612,18.485800,61104463.0
48,South Jordan,51057,22.263201,55642063.0
47,Saratoga Springs,18394,23.145201,55161851.0
0,American Fork,27413,11.256600,53236533.0
63,West Valley City,133796,35.799702,51992910.0


In [29]:
# 10 cities with largest difference of revenue
cities_q4_d20[['NAME', 'POPLASTCEN', 'AREA_SQMI', 'TOTAL_D20']].sort_values('TOTAL_D20', ascending=True).head(10)

,NAME,POPLASTCEN,AREA_SQMI,TOTAL_D20
32,North Salt Lake,16930,8.512150,-59910734.0
45,Salt Lake City,190749,110.734001,-48983396.0
43,Roosevelt,6260,6.757070,-3518293.0
57,Tooele City,32629,24.087099,-356201.0
30,Murray,47311,12.293700,-306705.0
28,Monticello,2053,3.349260,1218909.0
2,Blanding,3514,13.205500,1575504.0
9,Delta,3486,8.713920,1591043.0
35,Park City,7709,20.341700,1612619.0
31,Nephi,5544,4.741290,1861731.0


#### Some industries of interest:
- RETAIL-GASOLINE STATIONS (447)
- RETAIL-FOOD & BEVERAGE STORES (445)
- REAL ESTATE, RENTAL & LEASING (53)
- ARTS, ENTERTAINMENT AND RECREATION (71)
- ACCOMMODATION (721)